# Basketball Shots Analysis training notebook

## Installation and libraries

In [ ]:
%pip install ultralytics
%pip install roboflow

## Importing necessary libraries

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow

## Download the dataset

You can download the dataset from Roboflow [this link](https://universe.roboflow.com/cricket-qnb5l/basketball-xil7x/dataset/1). Make sure to unzip the file and place it in your working directory.

Or you can use the following code to download and unzip the dataset directly in your notebook:

In [ ]:
rf = Roboflow(api_key="Your_Roboflow_API_Key")
project = rf.workspace("cricket-qnb5l").project("basketball-xil7x")
version = project.version(1)
dataset = version.download("yolov8")

## Train the model
We will use a pre-trained model and fine-tune it on our basketball shots dataset using ultralytics YOLOv8.
we use yolov8m model for this task.

Before training make sure you have a GPU enabled runtime to speed up the training process.

In [ ]:
model = YOLO("yolov8m.pt") # here we are using the medium YOLOv8 model
epochs = 50
patience = 5

model.train(data=dataset.location, epochs=epochs , patience=patience)

## Download trained model

When the training is complete, you can download the trained model from "Runs" section in the left panel of the notebook interface. Look for the folder corresponding to your training run (e.g., "detect/train") and download the "weights" folder which contains the best model weights.

eg: runs/detect/train/weights/best.pt